In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-summarization/news_summary.csv


### Installing required Libraries

In [2]:
!pip install tensorboard
!pip install sentencepiece
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a269dc2301d10b06cc25ffdbd40cb93eac7505ce0d0601cc3ad24656b4415a8d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


### Importing Libraries

In [3]:
import torch
import pprint
import evaluate

from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)

from datasets import load_dataset

2024-04-29 18:13:47.358268: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 18:13:47.358373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 18:13:47.489971: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
pp = pprint.PrettyPrinter()

### Importing Dataset

In [5]:
df = pd.read_csv("/kaggle/input/news-summarization/news_summary.csv", encoding='latin1')

/tmp/ipykernel_35/2447125997.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,10,11,12,13,14,15,16,17,18,20,21,23,24,25,27,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,265,266,267,268,269,270,27

In [6]:
df = df[["Summary", "Text"]]

In [7]:
df

,Summary,Text
0,The Daman and Diu administration on Wednesday ...,The Administration of Union Territory Daman an...
1,"From her special numbers to TV?appearances, Bo...",Malaika Arora slammed an Instagram user who tr...
2,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
3,Hotels in Mumbai and other Indian cities are t...,Hotels in Maharashtra will train their staff t...
4,An alleged suspect in a kidnapping case was fo...,A 32-year-old man on Wednesday was found hangi...
...,...,...
4507,"Mumbai, Feb 23 (PTI) Fruit juice concentrate m...",Fruit juice concentrate maker Rasna is eyeing ...
4508,Former cricketer Sachin Tendulkar was spotted ...,Former Indian cricketer Sachin Tendulkar atten...
4509,"Aamir Khan, whose last film Dangal told the st...","Aamir Khan, while talking about reality shows ..."
4510,Maharahstra Power Minister Chandrashekhar Bawa...,The Maharashtra government has initiated an in...


## T5 Data Prep with Training Data Column Names

In [9]:
df = df.rename(columns={"Summary":"target_text", "Text":"source_text"})
df = df[['source_text', 'target_text']]

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4512 entries, 0 to 4511
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   source_text  4511 non-null   object
 1   target_text  4393 non-null   object
dtypes: object(2)
memory usage: 70.6+ KB


In [11]:
df.dropna(inplace=True)

In [12]:
df = df.astype({'source_text': str, 'target_text': str})
print(df.dtypes)

source_text    object
target_text    object
dtype: object


In [13]:
df.head()

,source_text,target_text
0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
3,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...
4,A 32-year-old man on Wednesday was found hangi...,An alleged suspect in a kidnapping case was fo...


In [14]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import Dataset, load_dataset

In [15]:
full_dataset = Dataset.from_pandas(df)

In [16]:
full_dataset = full_dataset.train_test_split(test_size=0.2, shuffle=True)

In [17]:
dataset_train = full_dataset['train']
dataset_valid = full_dataset['test']

In [18]:
print(dataset_train)
print(dataset_valid)

Dataset({
    features: ['source_text', 'target_text', '__index_level_0__'],
    num_rows: 3514
})
Dataset({
    features: ['source_text', 'target_text', '__index_level_0__'],
    num_rows: 879
})


## Using SimpleT5 for Model Training

### Configuration

In [19]:
MODEL = "t5-base"
BATCH_SIZE = 4
NUM_PROCS = 4
EPOCHS = 10
OUT_DIR ="result_t5base"
MAX_LENGTH=512

### Tokenization

In [20]:
from transformers import T5Tokenizer

In [21]:
tokenizer = T5Tokenizer.from_pretrained(MODEL)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
def preprocess_function(examples):
    inputs = [f"summarize: {article}" for article in examples["source_text"]]
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_LENGTH,
        truncation=True,
        padding="max_length"
    )
    
    targets = [summary for summary in examples["target_text"]]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=MAX_LENGTH,
            truncation=True,
            padding="max_length"
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenizer_train = dataset_train.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS
)

tokenized_valid = dataset_valid.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS
)

Map (num_proc=4):   0%|          | 0/3514 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text

Map (num_proc=4):   0%|          | 0/879 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text

### Model

In [23]:
model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

### ROUGE Metric

In [24]:
rouge = evaluate.load("rouge")

In [25]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)  # Corrected typo in variable name
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)  # Corrected typo in function name
    
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=[
            "rouge1",
            "rouge2",
            "rougeL"
        ]
    )
    
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    result = {k: round(v, 4) for k, v in result.items()}  # Corrected syntax to assign to result variable
    
    return result  # Added return statement


In [26]:
def preprocess_logits_for_metrics(logits, labels):
    
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

### Training

In [33]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=50,
    logging_dir=OUT_DIR,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=2,
    report_to="tensorboard",
    learning_rate=0.0001,
    dataloader_num_workers=4
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenizer_train,
    eval_dataset=tokenized_valid,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [34]:
history = trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Gen Len
1,2.364100,2.261965,0.522800,0.200600,0.398600,383.970400
2,2.408600,2.194570,0.528800,0.204600,0.404200,383.927200
3,2.266400,2.165240,0.531600,0.206700,0.406500,383.874900
4,2.184200,2.151594,0.535700,0.208300,0.408800,383.912400
5,2.128900,2.141923,0.536800,0.210100,0.410100,383.880500
6,2.005700,2.136505,0.537400,0.210300,0.410700,383.889600
7,2.042200,2.134662,0.539800,0.211100,0.411600,383.897600
8,2.040700,2.130030,0.540100,0.211400,0.411800,383.866900
9,1.910900,2.130827,0.541000,0.211400,0.412000,383.879400
10,2.099700,2.131778,0.540700,0.211700,0.412100,383.886200


In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_path = "/kaggle/input/t5-finetuned"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
!pip install rouge

In [9]:
model.to("cuda")

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [23]:
import torch
from rouge import Rouge


# Define the generated summary and the reference summary
reference_summary = '''Indian Football Witnesses 138% Growth In Women’s Player Registration In Last Two Years'''

# Initialize the ROUGE object
rouge = Rouge()

# Define the input text for GPT-2
input_text = '''Indian football has experienced a historic 138% surge in the registration of women players over the past two years, indicating a significant growth in the sport’s popularity and an increase in young female athletes pursuing football professionally.

According to data from the All India Football Federation (AIFF) Central Registration System (CRS), India now boasts 27,936 registered female footballers as of March 2024, compared to 11,724 in June 2022.

Kalyan Chaubey, President of AIFF, credited this growth to various initiatives implemented during the current season, including the introduction of the Indian Women’s League (IWL-2) as a second-tier competition.

The 2022-23 season of the IWL took place in Ahmedabad, featuring 16 teams in a single venue, with Gokulam Kerala FC clinching their third consecutive title. In the following season (2023-24), the format shifted to a ‘home-and-away’ structure, receiving widespread praise from clubs, players, and fans. Odisha FC emerged victorious, ending Gokulam Kerala FC’s three-year dominance and earning the opportunity to represent India in AFC continental competition.

The inaugural IWL-2 witnessed the participation of 15 clubs in the group stage, with six advancing to the final round scheduled for the next month in Kolkata.

India currently hosts 24 active State Leagues for women’s football, serving as the third tier of the pyramid and contributing to the sport’s widespread adoption across the country.

Chaubey said that over the past 16-18 months, they have taken incremental steps with a concentrated focus on women’s football. This current season marks a significant turning point for Indian football in multiple aspects.

He mentioned that they’re witnessing rapid growth in women’s football across India. Both players and clubs are benefiting from increased game time, driven by a rise in domestic matches, which has intensified competition. The prospect of participating in the AFC Women’s Cup, organized by the AFC, serves as a major incentive for Indian clubs today.

Looking ahead, the AIFF President emphasised that women’s football will receive more priority in the coming years, ensuring sustained development and progress in the sport.''' + '\nTL;DR:'

# Tokenize input text
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

# Generate predictions
output = model.generate(
        input_ids = input_ids["input_ids"],
        attention_mask = input_ids["attention_mask"],
        length_penalty=0.8,
        do_sample = True,
        max_new_tokens=30,
        num_beams=8
    )

# Decode the generated sequence
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

generated_text = output_text

In [26]:
# Calculate ROUGE for the generated and reference summaries
scores = rouge.get_scores(generated_text[24:], reference_summary)
# Print the results
print(scores)

[{'rouge-1': {'r': 0.16666666666666666, 'p': 0.1111111111111111, 'f': 0.1333333285333335}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.16666666666666666, 'p': 0.1111111111111111, 'f': 0.1333333285333335}}]


In [28]:
generated_text[24:]

'Indian football has experienced a historic 138% surge in the registration of women players over the past two years'

In [38]:
def summarize_text(text, model, max_length=512, num_beams=5):
    inputs = tokenizer.encode(
        "summarize" + text,
        return_tensors='pt',
        max_length=max_length,
        truncation=True
    )
    
    summary_ids = model.generate(
        inputs,
        max_length=24,
        num_beams=num_beams
    )
    
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
summarize_text()

### Saving Model

In [35]:
model.save_pretrained("T5_summarization")

In [36]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json')

In [37]:
from IPython.display import FileLink

file_path = "T5_summarization/model.safetensors"  # Specify the full path to the file
link = FileLink(file_path)
link


/kaggle/working/T5_summarization/model.safetensors